# Convolutional Autoencoder (CAE)
### Authors: Sergi Nogués and Gilles Salem

This is the second assignment for Advanced Concepts in Machine Learning
(MSc Artificial Intelligence, Maastricht University)

November 2021

[Github link](https://github.com/serginogues/convolutional_autoencoder)

CIFAR-10 dataset: https://www.cs.toronto.edu/~kriz/cifar.html